In [1]:
import pandas as pd
import numpy as np
import glob
from gplearn.genetic import SymbolicRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split
import torch
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from sklearn.metrics import r2_score, mean_squared_error

In [3]:
mens = []
womens = []
labels = pd.read_csv('../gender_labels.csv')
for s in glob.glob('../all_data_confounds_remove/*.csv'):
    person = int(s.split('/')[-1].split('_')[0])
    data = pd.read_csv(s)
    data = data.rolling(window=10).mean().dropna()
    if labels[labels['person']==person]['gender'].values[0]=='M':
        mens.append(data)
    else:
        womens.append(data)
mens = pd.concat(mens)
womens = pd.concat(womens)  

In [5]:
regions = mens.columns

In [6]:
model_woman = torch.nn.Sequential(
    torch.nn.Linear(47, 100),
    torch.nn.Softplus(),
    torch.nn.Linear(100, 50),
    torch.nn.Softplus(),
    torch.nn.Linear(50, 25),
    torch.nn.Tanh(),
    torch.nn.Linear(25, 1),
).to('cuda:2')

In [17]:
device='cuda:1'
data = womens
model_man = torch.nn.Sequential(
    torch.nn.Linear(47, 100),
    torch.nn.Softplus(),
    torch.nn.Linear(100, 50),
    torch.nn.Softplus(),
    torch.nn.Linear(50, 25),
    torch.nn.Tanh(),
    torch.nn.Linear(25, 1),
).to(device)
for region in regions:
    model_man.load_state_dict(torch.load('../models/pytorch_model_{0}_mens.pt'.format(region)))
    model_man.eval()

    X = data.drop([region], axis=1).values
    y = data[region].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=0)

    device='cuda:1'
    y_predict = np.ravel(model_man(torch.FloatTensor(X_test).to(device)).detach().cpu().numpy())
    print('region: {}, r2_score: {}'.format(region, np.round(r2_score(y_predict, y_test), 4)))

region: x0, r2_score: 0.1089
region: x1, r2_score: 0.4916
region: x2, r2_score: 0.0735
region: x3, r2_score: 0.3862
region: x4, r2_score: 0.045
region: x5, r2_score: 0.2468
region: x6, r2_score: 0.6172
region: x7, r2_score: 0.2201
region: x8, r2_score: 0.1125
region: x9, r2_score: 0.3716
region: x10, r2_score: 0.3468
region: x11, r2_score: 0.3955
region: x12, r2_score: -0.1936
region: x13, r2_score: -0.3176
region: x14, r2_score: -0.7044
region: x15, r2_score: 0.0747
region: x16, r2_score: 0.5984
region: x17, r2_score: 0.2063
region: x18, r2_score: 0.448
region: x19, r2_score: 0.4648
region: x20, r2_score: 0.4603
region: x21, r2_score: 0.3675
region: x22, r2_score: 0.3435
region: x23, r2_score: 0.5785
region: x24, r2_score: -0.1754
region: x25, r2_score: 0.3385
region: x26, r2_score: -0.8689
region: x27, r2_score: 0.1376
region: x28, r2_score: 0.1589
region: x29, r2_score: 0.4065
region: x30, r2_score: 0.5386
region: x31, r2_score: 0.4232
region: x32, r2_score: -0.0755
region: x33, r2_

In [12]:
error_regions = ['x14', 'x24', 'x26', 'x28', 'x32', 'x33', 'x34', 'x36', 'x37', 'x43', 'x44', 'x47']

In [14]:
len(error_regions)

12